## 메모리 초과
- 가로, 세로가 2^15까지 가능하다

In [9]:
import sys 
sys.stdin = open('test.txt', 'r')
sys.setrecursionlimit(int(1e6))
input = sys.stdin.readline

# N(2^N * 2^N 행렬)이 주어졌을 때, r행 c열을 몇 번째르 방문하는가?
# 0번째부터 방문함
N, r, c = map(int, input().split())
graph = [[-1] * (2**N) for _ in range(2**N)]

# 방문 순서 : 주어진 행렬을 2^N-1 * 2^N-1로 쪼갠 다음, 왼쪽 위 - 오른쪽 위 - 왼쪽 아래 - 오른쪽 아래 순서대로 방문함
# 그러려면 N 값 외에도 각 행렬의 왼쪽 위 위치를 알아야 할 듯?
# 이전 좌측 상단 좌표 를 기준, +{ (0, 0) / (2^(N-1), 0) / (0, 2^(N-1)), (2^(N-1), 2^(N-1))}

order = 0

def funcZ(N, row, col):
    global order

    # 재귀 종료 조건
    if N == 0:
        graph[row][col] = order
        order += 1
    
    else:
        funcZ(N-1, row, col)
        funcZ(N-1, row, col + 2**(N-1))
        funcZ(N-1, row + 2**(N-1), col)
        funcZ(N-1, row + 2**(N-1), col + 2**(N-1))


funcZ(N, 0, 0)
print(graph[r][c])

786432


## 다른 답변 참고
- 그래프는 그리지 않는 게 맞다! 2^30의 그래프를 그릴 수는 없으니까!
- 그래프를 그리지 않고, r행 c열의 값을 어떻게 알 수 있는가?

### 1. 분할 정복
- N = 3일 때 4분할할 경우, 분할된 각 행렬의 좌측상단 값은4^2(2^4)의 배수임
- N = 2일 때 4분할할 경우, 분할된 각 행렬의 좌측상단 값은 4^1의 배수임
- N = 1일 때 4분할할 경우, 분할된 각 행렬의 좌측상단 값은 4^0의 배수임

- 그리고 분할된 행렬 중, r행 c열이 어떤 행렬에 속하는지는 자명함

In [13]:
import sys 
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N, r, c = map(int, input().split())

ans = 0 

# 현재 행렬의 좌측 상단 row, col 값
base_row = 0
base_col = 0

while N > 0:
    # 2^N * 2^N 행렬에서 절반으로 자르는 값을 설정
    value = 2 ** (N - 1)
    half_row = base_row + value
    half_col = base_col + value

    # 조건은 좌상 / 우상 / 좌하 / 우하 순서
    # r, c가 둘 다 작다 / 둘 중 하나만 크거나 같다[(0, col), (row, 0)] 순서 / 둘 다 크거나 같다
    if r < half_row and c < half_col:
        ans += 4 ** (N - 1) * 0
        base_row += 0
        base_col += 0
    elif r < half_row and c >= half_col:
        ans += 4 ** (N - 1) * 1
        base_row += 0
        base_col += value
    elif r >= half_row and c < half_col:
        ans += 4 ** (N - 1) * 2
        base_row += value
        base_col += 0
    else:
        ans += 4 ** (N - 1) * 3
        base_row += value
        base_col += value
    N -= 1

print(ans)

262143


### 2. 재귀
- `arr[a][b] = c` 일 때, `arr[2a][2b] = 4c` 라는 규칙이 있기는 하다
- 그렇게 접근하면 `4 * funcZ(N-1, r//2, c//2)`로 재귀식을 짤 수 있기는 한데..
- 그러면 나머지는 어떻게 하지?

---
#### 홀수 인덱스의 처리
- 저 논리대로면 결국 걸리는 지점은 홀수 인덱스가 있을 때란 말임?
- 가장 작은 2x2 행렬에서, **홀수 인덱스가 있는 경우는 왼쪽 상단의 칸을 제외한 모든 경우**임
- 그러면 좌측 상단의 값으로 표준화를 해서 줄일 수도 있겠다
- 즉, 모든 2x2 행렬에서 우측 상단은 col 값이 홀수, 좌측 하단은 row 값이 홀수, 우측 하단은 row, col 모두 홀수임
- 그렇게 생각하면 표준화를 할 때 아래처럼 이동시키면 됨
  - row 값은 2로 나눈 나머지 값 * 2
  - col 값은 2로 나눈 나머지 값

In [18]:
import sys 
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N, r, c = map(int, input().split())

def funcZ(N, r, c):
    if N == 0:
        return 0

    return (2 * (r % 2) + c % 2) + 4 * funcZ(N - 1, r // 2, c // 2)
    
print(funcZ(N, r, c))

11